<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LlamaIndex/3-State02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-parse llama-index-embeddings-openai llama-index-readers-whisper llama-index-tools-yahoo-finance

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec
from llama_index.core.workflow import Context

In [3]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [4]:
llm = OpenAI(model="gpt-4o-mini")

async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"

In [5]:
workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

In [6]:
import asyncio

ctx = Context(workflow)

# "What's my name?" 실행
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response))

Your name has been set to "unset."


In [7]:
# "My name is Laurie" 실행
response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response2))

Your name has been updated to "Laurie."


In [8]:
# state에서 값 직접 가져오기
state = await ctx.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Laurie


In [ ]:
# async def main():
#     ctx = Context(workflow)

#     # check if it knows a name before setting it
#     response = await workflow.run(user_msg="What's my name?", ctx=ctx)
#     print(str(response))

#     # set the name using a tool
#     response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
#     print(str(response2))

#     # retrieve the value from the state directly
#     state = await ctx.get("state")
#     print("Name as stored in state: ",state["name"])

# if __name__ == "__main__":
#     import asyncio
#     asyncio.run(main())